# Import library

In [1]:
from sklearn import preprocessing, svm, tree
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, recall_score, f1_score, precision_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import numpy as np
import pandas as pd

# Read data

In [2]:
df = pd.read_csv('pd_speech_features.csv')
df.drop(['id'], 1, inplace=True)

# Summary of data

In [3]:
df.head()

,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,locAbsJitter,...,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
0,1,0.85247,0.71826,0.57227,240,239,0.008064,0.000087,0.00218,0.000018,...,1.5620,2.6445,3.8686,4.2105,5.1221,4.4625,2.6202,3.0004,18.9405,1
1,1,0.76686,0.69481,0.53966,234,233,0.008258,0.000073,0.00195,0.000016,...,1.5589,3.6107,23.5155,14.1962,11.0261,9.5082,6.5245,6.3431,45.1780,1
2,1,0.85083,0.67604,0.58982,232,231,0.008340,0.000060,0.00176,0.000015,...,1.5643,2.3308,9.4959,10.7458,11.0177,4.8066,2.9199,3.1495,4.7666,1
3,0,0.41121,0.79672,0.59257,178,177,0.010858,0.000183,0.00419,0.000046,...,3.7805,3.5664,5.2558,14.0403,4.2235,4.6857,4.8460,6.2650,4.0603,1
4,0,0.32790,0.79782,0.53028,236,235,0.008162,0.002669,0.00535,0.000044,...,6.1727,5.8416,6.0805,5.7621,7.7817,11.6891,8.2103,5.0559,6.1164,1


In [4]:
df.tail()

,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,locAbsJitter,...,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
751,0,0.80903,0.56355,0.28385,417,416,0.004627,0.000052,0.00064,0.000003,...,3.0706,3.0190,3.1212,2.4921,3.5844,3.5400,3.3805,3.2003,6.8671,0
752,0,0.16084,0.56499,0.59194,415,413,0.004550,0.000220,0.00143,0.000006,...,1.9704,1.7451,1.8277,2.4976,5.2981,4.2616,6.3042,10.9058,28.4170,0
753,0,0.88389,0.72335,0.46815,381,380,0.005069,0.000103,0.00076,0.000004,...,51.5607,44.4641,26.1586,6.3076,2.8601,2.5361,3.5377,3.3545,5.0424,0
754,0,0.83782,0.74890,0.49823,340,339,0.005679,0.000055,0.00092,0.000005,...,19.1607,12.8312,8.9434,2.2044,1.9496,1.9664,2.6801,2.8332,3.7131,0
755,0,0.81304,0.76471,0.46374,340,339,0.005676,0.000037,0.00078,0.000004,...,62.9927,21.8152,9.2457,4.8555,3.0551,3.0415,4.0116,2.6217,3.1527,0


In [5]:
count_class_0 = (df['class']== 0).sum()
count_class_1 = (df['class']== 1).sum()
print(count_class_0, count_class_1)

192 564


In [6]:
X = np.array(df.drop(['class'], 1))
y = np.array(df['class'])
print(X, y)

[[ 1.       0.85247  0.71826 ...  2.6202   3.0004  18.9405 ]
 [ 1.       0.76686  0.69481 ...  6.5245   6.3431  45.178  ]
 [ 1.       0.85083  0.67604 ...  2.9199   3.1495   4.7666 ]
 ...
 [ 0.       0.88389  0.72335 ...  3.5377   3.3545   5.0424 ]
 [ 0.       0.83782  0.7489  ...  2.6801   2.8332   3.7131 ]
 [ 0.       0.81304  0.76471 ...  4.0116   2.6217   3.1527 ]] [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1
 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0
 0 1 1 1 0 0 0 0 0 0 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 1 1 1 0 0 0 0 0 0 0
 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0
 1 1 1 0 0 0 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0

# Normalizing Data

In [7]:
Z = np.divide((X - X.mean(0)), X.std(0))

In [8]:
Z

array([[ 0.96874225,  0.62764391,  0.2561442 , ..., -0.775137  ,
        -0.81472704, -0.36659507],
       [ 0.96874225,  0.12161952, -0.08043329, ..., -0.52664699,
        -0.58297219,  0.40039616],
       [ 0.96874225,  0.61795018, -0.34983882, ..., -0.75606253,
        -0.8043897 , -0.7809355 ],
       ...,
       [-1.03226633,  0.81336154,  0.3292009 , ..., -0.71674252,
        -0.79017671, -0.77287314],
       [-1.03226633,  0.54105055,  0.69591966, ..., -0.77132466,
        -0.82631929, -0.81173208],
       [-1.03226633,  0.3945807 ,  0.92284035, ..., -0.68658105,
        -0.84098293, -0.82811405]])

In [9]:
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

# Create Decision Tree Model

In [10]:
dt = tree.DecisionTreeClassifier( criterion='entropy', max_depth=5, random_state=42 )

model = dt
scores = cross_val_score(model, Z, y, cv=5, scoring='accuracy')
y_pred = cross_val_predict(model, Z, y, cv=5)

acc = scores.mean()
conf_mat = confusion_matrix(y, y_pred)

# Result

In [11]:
print('accuracy:', acc)
print('f1_score:', f1_score(y, y_pred, average='micro'))
print('recall_score:',recall_score(y, y_pred, average='micro'))
print('precision_score:',precision_score(y, y_pred, average='micro'))
print('conf_mat:\n', conf_mat)
print(classification_report(y, y_pred))

accuracy: 0.7659637504356918
f1_score: 0.765873015873016
recall_score: 0.7658730158730159
precision_score: 0.7658730158730159
conf_mat:
 [[110  82]
 [ 95 469]]
              precision    recall  f1-score   support

           0       0.54      0.57      0.55       192
           1       0.85      0.83      0.84       564

    accuracy                           0.77       756
   macro avg       0.69      0.70      0.70       756
weighted avg       0.77      0.77      0.77       756



# Random Forest Model

In [12]:
df = pd.read_csv('pd_speech_features.csv')
df.drop(['id'], 1, inplace=True)

In [13]:
Z = np.divide((X - X.mean(0)), X.std(0))

In [14]:
rd = RandomForestClassifier(n_estimators=100, criterion='entropy')
model = rd
scores = cross_val_score(model, Z, y, cv=5, scoring='accuracy')
y_pred = cross_val_predict(model, Z, y, cv=5)

acc = scores.mean()
conf_mat = confusion_matrix(y, y_pred)

In [15]:
print('accuracy:', acc)
print('f1_score:', f1_score(y, y_pred, average='micro'))
print('recall_score:',recall_score(y, y_pred, average='micro'))
print('precision_score:',precision_score(y, y_pred, average='micro'))
print('conf_mat:\n', conf_mat)
print(classification_report(y, y_pred))

accuracy: 0.842662948762635
f1_score: 0.8359788359788359
recall_score: 0.8359788359788359
precision_score: 0.8359788359788359
conf_mat:
 [[ 90 102]
 [ 22 542]]
              precision    recall  f1-score   support

           0       0.80      0.47      0.59       192
           1       0.84      0.96      0.90       564

    accuracy                           0.84       756
   macro avg       0.82      0.71      0.74       756
weighted avg       0.83      0.84      0.82       756



# XGBoost Model

In [16]:
df = pd.read_csv('pd_speech_features.csv')
df.drop(['id'], 1, inplace=True)

In [17]:
Z = np.divide((X - X.mean(0)), X.std(0))

In [18]:
XGB = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=100, criterion='friedman_mse')

model = XGB
scores = cross_val_score(model, Z, y, cv=5, scoring='accuracy')
y_pred = cross_val_predict(model, Z, y, cv=5)

acc = scores.mean()
conf_mat = confusion_matrix(y, y_pred)

In [19]:
print('accuracy:', acc)
print('f1_score:', f1_score(y, y_pred, average='micro'))
print('recall_score:',recall_score(y, y_pred, average='micro'))
print('precision_score:',precision_score(y, y_pred, average='micro'))
print('conf_mat:\n', conf_mat)
print(classification_report(y, y_pred))

accuracy: 0.8399878006273964
f1_score: 0.8359788359788359
recall_score: 0.8359788359788359
precision_score: 0.8359788359788359
conf_mat:
 [[101  91]
 [ 33 531]]
              precision    recall  f1-score   support

           0       0.75      0.53      0.62       192
           1       0.85      0.94      0.90       564

    accuracy                           0.84       756
   macro avg       0.80      0.73      0.76       756
weighted avg       0.83      0.84      0.83       756



# SVM Model with rbf kernel

In [20]:
df = pd.read_csv('pd_speech_features.csv')
df.drop(['id'], 1, inplace=True)

In [21]:
Z = np.divide((X - X.mean(0)), X.std(0))

In [22]:
svm = svm.SVC(C=1.0, kernel='rbf', degree=3, gamma='scale')

model = svm
scores = cross_val_score(model, Z, y, cv=5, scoring='accuracy')
y_pred = cross_val_predict(model, Z, y, cv=5)

acc = scores.mean()
conf_mat = confusion_matrix(y, y_pred)

In [23]:
print('accuracy:', acc)
print('f1_score:', f1_score(y, y_pred, average='micro'))
print('recall_score:',recall_score(y, y_pred, average='micro'))
print('precision_score:',precision_score(y, y_pred, average='micro'))
print('conf_mat:\n', conf_mat)
print(classification_report(y, y_pred))

accuracy: 0.8360055768560473
f1_score: 0.8359788359788359
recall_score: 0.8359788359788359
precision_score: 0.8359788359788359
conf_mat:
 [[ 82 110]
 [ 14 550]]
              precision    recall  f1-score   support

           0       0.85      0.43      0.57       192
           1       0.83      0.98      0.90       564

    accuracy                           0.84       756
   macro avg       0.84      0.70      0.73       756
weighted avg       0.84      0.84      0.82       756



# SVM Model with linear kernel

In [24]:
from sklearn import preprocessing, svm, tree
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, recall_score, f1_score, precision_score
from sklearn.model_selection import cross_val_score, cross_val_predict
import numpy as np
import pandas as pd

In [27]:
svm1 = svm.SVC(C=1.0, kernel='linear', gamma='scale')

model = svm1
scores = cross_val_score(model, Z, y, cv=5, scoring='accuracy')
y_pred = cross_val_predict(model, Z, y, cv=5)

acc = scores.mean()
conf_mat = confusion_matrix(y, y_pred)

In [28]:
print('accuracy:', acc)
print('f1_score:', f1_score(y, y_pred, average='micro'))
print('recall_score:',recall_score(y, y_pred, average='micro'))
print('precision_score:',precision_score(y, y_pred, average='micro'))
print('conf_mat:\n', conf_mat)
print(classification_report(y,y_pred))

accuracy: 0.7501045660508889
f1_score: 0.75
recall_score: 0.75
precision_score: 0.75
conf_mat:
 [[102  90]
 [ 99 465]]
              precision    recall  f1-score   support

           0       0.51      0.53      0.52       192
           1       0.84      0.82      0.83       564

    accuracy                           0.75       756
   macro avg       0.67      0.68      0.68       756
weighted avg       0.75      0.75      0.75       756

